In [405]:
import pandas as pd
from variables import *

In [406]:
provinces = ['Venezia', 'Verona', 'Vicenza', 'Padova', 'Treviso', 'Rovigo', 'Belluno']
provinces_map = {'VE': 'Venezia', 'VR': 'Verona', 'VI': 'Vicenza', 'PD': 'Padova', 'TV': 'Treviso', 'RO': 'Rovigo', 'BL': 'Belluno'}

In [407]:
ISTAT_VENETO_CODE = 5
MUNICIPALITY_NAME_COLUMN = 'Municipality Name'
ISTAT_CODE_COLUMN = 'Istat code'
PROVINCE_COLUMN = 'Province'
SURFACE_COLUMN = 'Surface (km2)'

In [408]:
gdp_df = pd.read_csv('./data/Redditi_e_principali_variabili_IRPEF_su_base_comunale_CSV_2019.csv', sep=';', index_col=False)
gdp_df.head()

,Anno di imposta,Codice catastale,Codice Istat Comune,Denominazione Comune,Sigla Provincia,Regione,Codice Istat Regione,Numero contribuenti,Reddito da fabbricati - Frequenza,Reddito da fabbricati - Ammontare in euro,...,Reddito complessivo da 15000 a 26000 euro - Ammontare in euro,Reddito complessivo da 26000 a 55000 euro - Frequenza,Reddito complessivo da 26000 a 55000 euro - Ammontare in euro,Reddito complessivo da 55000 a 75000 euro - Frequenza,Reddito complessivo da 55000 a 75000 euro - Ammontare in euro,Reddito complessivo da 75000 a 120000 euro - Frequenza,Reddito complessivo da 75000 a 120000 euro - Ammontare in euro,Reddito complessivo oltre 120000 euro - Frequenza,Reddito complessivo oltre 120000 euro - Ammontare in euro,
0,2019,A001,28001,ABANO TERME,PD,Veneto,5,14954,7477.0,10358625.0,...,100630146,3472.0,121942029.0,438.0,27886866.0,396.0,36808564.0,187.0,38228141.0,NaN
1,2019,A004,98001,ABBADIA CERRETO,LO,Lombardia,3,205,86.0,66213.0,...,1614434,46.0,1489603.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,2019,A005,97001,ABBADIA LARIANA,LC,Lombardia,3,2441,1193.0,1723810.0,...,17001505,671.0,23138246.0,74.0,4698905.0,48.0,4328711.0,31.0,6397476.0,NaN
3,2019,A006,52001,ABBADIA SAN SALVATORE,SI,Toscana,9,4876,2635.0,2827574.0,...,37044603,796.0,27215546.0,68.0,4342264.0,32.0,2960972.0,9.0,1435917.0,NaN
4,2019,A007,95001,ABBASANTA,OR,Sardegna,20,1731,875.0,766745.0,...,9187754,433.0,14540500.0,24.0,1572949.0,10.0,908389.0,NaN,NaN,NaN


In [409]:
gdp_df = gdp_df[gdp_df['Codice Istat Regione'] == ISTAT_VENETO_CODE]

In [410]:
gdp_df.reset_index(inplace=True)

In [411]:
gdp_df.shape

(563, 52)

In [412]:
gdp_df['Reddito imponibile - Ammontare in euro'].isna().sum()

0

In [413]:
gdp_df['Reddito imponibile - Ammontare in euro'].isna().sum()

0

In [414]:
gdp_df['Numero contribuenti'].isna().sum()

0

In [415]:
municipality_info_df = gdp_df[['Denominazione Comune', 'Codice Istat Comune', 'Sigla Provincia']]

In [416]:
municipality_info_df = municipality_info_df.rename(columns={
    'Denominazione Comune': MUNICIPALITY_NAME_COLUMN, 
    'Codice Istat Comune': ISTAT_CODE_COLUMN, 
    'Sigla Provincia': PROVINCE_COLUMN
})

municipality_info_df[MUNICIPALITY_NAME_COLUMN] = municipality_info_df[MUNICIPALITY_NAME_COLUMN].str.lower()

In [417]:
municipality_info_df.head()

,Municipality Name,Istat code,Province
0,abano terme,28001,PD
1,adria,29001,RO
2,affi,23001,VR
3,agna,28002,PD
4,agordo,25001,BL


In [418]:
municipality_info_df[PER_UNIT_GDP] = gdp_df['Reddito imponibile - Ammontare in euro'] / gdp_df['Numero contribuenti']

In [419]:
municipality_info_df.head()

,Municipality Name,Istat code,Province,Per unit GDP
0,abano terme,28001,PD,22672.425104
1,adria,29001,RO,19309.673843
2,affi,23001,VR,20106.002743
3,agna,28002,PD,17973.677643
4,agordo,25001,BL,23715.945362


In [420]:
#surfaces_df = pd.read_excel('./data/Dati comunali e provinciali.xlsx', 'Dati comunali')
surfaces_df = pd.read_excel('./data/2338_Superficie_territoriale_in_kmq.xls')

#surfaces_df = surfaces_df[surfaces_df['Codice Regione'] == ISTAT_VENETO_CODE]

In [421]:
surfaces_df = surfaces_df[['Codice Istat comune', 'Superficie Kmq']]

In [422]:
#

In [423]:
surfaces_df.head()

,Codice Istat comune,Superficie Kmq
0,23001,9.85
1,23002,28.22
2,23003,13.46
3,23004,18.82
4,23005,26.86


In [424]:
# municipality_info_df[SURFACE_COLUMN] = 0

In [425]:
municipality_info_df = pd.merge(
    municipality_info_df, 
    surfaces_df, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='Codice Istat comune', 
    how = 'left'
)
municipality_info_df = municipality_info_df.rename(columns={
    'Superficie Kmq': SURFACE_COLUMN,
})
municipality_info_df.drop('Codice Istat comune', axis=1, inplace=True)

In [426]:
municipality_info_df[municipality_info_df[SURFACE_COLUMN].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2)
551,quero vas,25070,BL,18012.158351,NaN
552,longarone,25071,BL,19326.091770,NaN
553,val di zoldo,25073,BL,15285.987690,NaN
554,alpago,25072,BL,19631.981083,NaN
555,val liona,24123,VI,20148.390223,NaN
556,barbarano mossano,24124,VI,18563.440208,NaN
557,borgo veneto,28107,PD,17286.636487,NaN
558,borgo valbelluna,25074,BL,19460.162789,NaN
559,pieve del grappa,26096,TV,18093.788296,NaN
560,valbrenta,24125,VI,17956.865122,NaN


In [427]:
municipality_info_df.loc[551, SURFACE_COLUMN] = 45.92
municipality_info_df.loc[552, SURFACE_COLUMN] = 122.36
municipality_info_df.loc[553, SURFACE_COLUMN] = 141.65
municipality_info_df.loc[554, SURFACE_COLUMN] = 80.34
municipality_info_df.loc[555, SURFACE_COLUMN] = 27.85
municipality_info_df.loc[556, SURFACE_COLUMN] = 33.486
municipality_info_df.loc[557, SURFACE_COLUMN] = 39.171
municipality_info_df.loc[558, SURFACE_COLUMN] = 167.69
municipality_info_df.loc[559, SURFACE_COLUMN] = 37.34
municipality_info_df.loc[560, SURFACE_COLUMN] = 93.37
municipality_info_df.loc[561, SURFACE_COLUMN] = 19.40
municipality_info_df.loc[562, SURFACE_COLUMN] = 61.19

In [428]:
municipality_info_df[municipality_info_df[SURFACE_COLUMN].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2)


In [429]:
df_density = pd.read_excel('./data/05_Veneto_Allegato-statistico.xlsx', 'Appendice 2',  skiprows = 10)
df_density = df_density[['ProCom', 'Densità']]
df_density.head()

,ProCom,Densità
0,23001,239.1
1,23002,186.3
2,23003,168.2
3,23004,336.2
4,23005,97.9


In [430]:
municipality_info_df = pd.merge(
    municipality_info_df, 
    df_density, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='ProCom', 
    how = 'left'
)
municipality_info_df = municipality_info_df.rename(columns={
    'Densità': POPULATION_DENSITY,
})
municipality_info_df.drop('ProCom', axis=1, inplace=True)

In [431]:
municipality_info_df[municipality_info_df[POPULATION_DENSITY].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density
558,borgo valbelluna,25074,BL,19460.162789,167.69,NaN
559,pieve del grappa,26096,TV,18093.788296,37.34,NaN
560,valbrenta,24125,VI,17956.865122,93.37,NaN
561,colceresa,24126,VI,19783.550410,19.40,NaN
562,lusiana conco,24127,VI,17003.585115,61.19,NaN


In [432]:
municipality_info_df.loc[558, POPULATION_DENSITY] = 80.15
municipality_info_df.loc[559, POPULATION_DENSITY] = 175.74
municipality_info_df.loc[560, POPULATION_DENSITY] = 53.28
municipality_info_df.loc[561, POPULATION_DENSITY] = 307.63
municipality_info_df.loc[562, POPULATION_DENSITY] = 76.32

In [433]:
municipality_info_df[municipality_info_df[POPULATION_DENSITY].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density


In [434]:
municipality_info_df.head()

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density
0,abano terme,28001,PD,22672.425104,21.57,946.5
1,adria,29001,RO,19309.673843,113.52,170.6
2,affi,23001,VR,20106.002743,9.85,239.1
3,agna,28002,PD,17973.677643,18.83,171.8
4,agordo,25001,BL,23715.945362,23.67,173.4


# Road Density

In [435]:
"""df_streets = pd.concat(
    map(
        pd.read_csv, 
        ['./data/Regione-Veneto----Grafo-strade-provincia-{}-elemento.csv'.format(p) for p in provinces]
    ), ignore_index=True
)"""
li = []
for p in provinces:
    print(p)
    df = pd.read_csv('./data/Regione-Veneto----Grafo-strade-provincia-{}-elemento.csv'.format(p), 
                     sep=';',  encoding='cp1252')
    #df = df[['ISTAT_ENTE', 'PERIMETRO']]
    #df = df.groupby('ISTAT_ENTE').sum()
    li.append(df)

df_streets = pd.concat(li, axis=0, ignore_index=True)

Venezia
Verona
Vicenza


C:\Users\riccardo\AppData\Local\Temp\ipykernel_12976\4181778230.py:10: DtypeWarning: Columns (0,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/Regione-Veneto----Grafo-strade-provincia-{}-elemento.csv'.format(p),


Padova
Treviso
Rovigo
Belluno


In [436]:
df_streets.head()

,OBJECTID,ID_ELEM,ID_PERC,ORIG,NOME,COD_ENTE,STATO,TIPO_ELE,SEDE,ID_SEDE,...,C_TF,FLG_ST,NODO_INI,NODO_FIN,VISIBILE,PERCORSOAM,CLAS_AMM,ISTAT_ENTE,PERIMETRO,ORIG_FID
0,0.0,580000059.0,1250.0,2.0,A 4 AUTOSTRADA DELLA SERENISSIMA MI-VE,5.0,1.0,1.0,2.0,NaN,...,1.0,0.0,58.0,59.0,1.0,A 4 AUTOSTRADA DELLA SERENISSIMA MI-VE,7.0,0.0,15.18,3.0
1,0.0,600000061.0,46198.0,2.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,5.0,1.0,2.0,5.0,NaN,...,1.0,0.0,60.0,61.0,1.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,7.0,0.0,60.09,4.0
2,0.0,600000061.0,46198.0,2.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,5.0,1.0,2.0,5.0,NaN,...,1.0,0.0,60.0,61.0,1.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,7.0,0.0,119.02,4.0
3,0.0,600000061.0,46198.0,2.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,5.0,1.0,2.0,5.0,NaN,...,1.0,0.0,60.0,61.0,1.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,7.0,0.0,190.54,4.0
4,0.0,620000063.0,46198.0,2.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,5.0,1.0,2.0,5.0,NaN,...,1.0,0.0,62.0,63.0,1.0,A 27 AUTOSTRADA D'ALEMAGNA VE-BL,7.0,0.0,14.08,5.0


In [437]:
df_streets.groupby('ISTAT_ENTE')['PERIMETRO'].sum()

ISTAT_ENTE
0.0        1088010.11
23001.0      17613.00
23002.0      34176.25
23003.0      20993.03
23004.0      25807.51
              ...    
29048.0      37204.71
29049.0      33919.54
29050.0      15228.08
29051.0      27211.04
29052.0     120086.56
Name: PERIMETRO, Length: 582, dtype: float64

In [438]:
# 0.003 is equivalent to the km of a street
df_streets = df_streets.groupby('ISTAT_ENTE')['PERIMETRO'].sum() #* 0.003
df_streets.reset_index()

,ISTAT_ENTE,PERIMETRO
0,0.0,1088010.11
1,23001.0,17613.00
2,23002.0,34176.25
3,23003.0,20993.03
4,23004.0,25807.51
...,...,...
577,29048.0,37204.71
578,29049.0,33919.54
579,29050.0,15228.08
580,29051.0,27211.04


In [439]:
# 0.001 is equivalent to the km of a street width and the streets are transformed in kmeters from meters again * 0.001
#scala 1 100000

In [440]:
municipality_info_df = pd.merge(
    municipality_info_df, 
    df_streets, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='ISTAT_ENTE', 
    how = 'left'
)

municipality_info_df[ROAD_DENSITY] = municipality_info_df.apply(
    lambda x: (x['PERIMETRO'] * 10000 * (0.001 * 0.001))/x[SURFACE_COLUMN], axis=1
)

municipality_info_df.drop('PERIMETRO', axis=1, inplace=True)

In [441]:
municipality_info_df[ROAD_DENSITY].max()

65.9423154185022

In [442]:
municipality_info_df[municipality_info_df[ROAD_DENSITY].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density
551,quero vas,25070,BL,18012.158351,45.920,68.50,NaN
552,longarone,25071,BL,19326.091770,122.360,42.50,NaN
553,val di zoldo,25073,BL,15285.987690,141.650,22.30,NaN
554,alpago,25072,BL,19631.981083,80.340,85.30,NaN
555,val liona,24123,VI,20148.390223,27.850,110.70,NaN
556,barbarano mossano,24124,VI,18563.440208,33.486,191.80,NaN
557,borgo veneto,28107,PD,17286.636487,39.171,178.70,NaN
558,borgo valbelluna,25074,BL,19460.162789,167.690,80.15,NaN
559,pieve del grappa,26096,TV,18093.788296,37.340,175.74,NaN
560,valbrenta,24125,VI,17956.865122,93.370,53.28,NaN


In [443]:
municipality_info_df[ROAD_DENSITY].fillna(municipality_info_df[ROAD_DENSITY].mean(), inplace=True)

In [444]:
municipality_info_df[municipality_info_df[ROAD_DENSITY].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density


## Elevation and Slope

In [445]:
elevation_df = pd.read_excel('./data/Elab_Altimetrie_DEM.xlsx')
elevation_df.head()

,COD_REG,COD_PRO,PRO_COM,NOME,SUPERFICIE (in KMQ),ALT_MIN,ALT_MAX,RANGE,MEDIA,MEDIANA,STD
0,12,58,58120,Fiumicino,213.8944,-3,164,167,31.688450,5,39.686070
1,5,29,29001,Adria,113.3900,-3,8,11,1.057863,1,1.244239
2,5,29,29030,Loreo,39.8442,-3,7,10,0.464185,0,0.770743
3,5,29,29052,Porto Viro,133.7686,-3,6,9,0.555705,1,0.553657
4,8,38,38017,Ostellato,173.3434,-3,8,11,1.784421,1,0.977924


In [446]:
#elevation_df[elevation_df['PRO_COM'] == 28001]

In [447]:
elevation_df = elevation_df[elevation_df['COD_REG'] == ISTAT_VENETO_CODE]

elevation_df = elevation_df.rename(columns={
    'MEDIA': ELEVATION,
    'RANGE': SLOPE,
})
elevation_df = elevation_df[['PRO_COM', SLOPE, ELEVATION]]

In [448]:
municipality_info_df = pd.merge(
    municipality_info_df, 
    elevation_df, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='PRO_COM', 
    how = 'left'
)

municipality_info_df.drop('PRO_COM', axis=1, inplace=True)

In [449]:
municipality_info_df[municipality_info_df[ELEVATION].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation
551,quero vas,25070,BL,18012.158351,45.920,68.50,15.161868,NaN,NaN
552,longarone,25071,BL,19326.091770,122.360,42.50,15.161868,NaN,NaN
553,val di zoldo,25073,BL,15285.987690,141.650,22.30,15.161868,NaN,NaN
554,alpago,25072,BL,19631.981083,80.340,85.30,15.161868,NaN,NaN
555,val liona,24123,VI,20148.390223,27.850,110.70,15.161868,NaN,NaN
556,barbarano mossano,24124,VI,18563.440208,33.486,191.80,15.161868,NaN,NaN
557,borgo veneto,28107,PD,17286.636487,39.171,178.70,15.161868,NaN,NaN
558,borgo valbelluna,25074,BL,19460.162789,167.690,80.15,15.161868,NaN,NaN
559,pieve del grappa,26096,TV,18093.788296,37.340,175.74,15.161868,NaN,NaN
560,valbrenta,24125,VI,17956.865122,93.370,53.28,15.161868,NaN,NaN


* data filled from: https://www.cittaeborghi.it/it

In [450]:
municipality_info_df.loc[551, ELEVATION] = 288; municipality_info_df.loc[551, SLOPE] = 1625 - 175
municipality_info_df.loc[552, ELEVATION] = 473 ; municipality_info_df.loc[552, SLOPE] = 2502 - 400 
municipality_info_df.loc[553, ELEVATION] = 1177; municipality_info_df.loc[553, SLOPE] = 3210 - 593
municipality_info_df.loc[554, ELEVATION] = 690; municipality_info_df.loc[554, SLOPE] = 2471 - 380
municipality_info_df.loc[555, ELEVATION] = 36; municipality_info_df.loc[555, SLOPE] = 320 - 15
municipality_info_df.loc[556, ELEVATION] = 72; municipality_info_df.loc[556, SLOPE] = 445 - 15
municipality_info_df.loc[557, ELEVATION] = 12; municipality_info_df.loc[557, SLOPE] = 22 - 7
municipality_info_df.loc[558, ELEVATION] = 362; 
municipality_info_df.loc[559, ELEVATION] = 300
municipality_info_df.loc[560, ELEVATION] = 160
municipality_info_df.loc[561, ELEVATION] = 150
municipality_info_df.loc[562, ELEVATION] = 830

municipality_info_df[SLOPE].fillna(municipality_info_df[SLOPE].mean(), inplace=True)

In [451]:
municipality_info_df[municipality_info_df[ELEVATION].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation


In [452]:
municipality_info_df

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation
0,abano terme,28001,PD,22672.425104,21.57,946.50,31.767918,70.000000,13.535090
1,adria,29001,RO,19309.673843,113.52,170.60,15.708153,11.000000,1.057863
2,affi,23001,VR,20106.002743,9.85,239.10,17.881218,248.000000,231.337300
3,agna,28002,PD,17973.677643,18.83,171.80,10.466909,3.000000,2.337880
4,agordo,25001,BL,23715.945362,23.67,173.40,6.942357,2329.000000,1267.740000
...,...,...,...,...,...,...,...,...,...
558,borgo valbelluna,25074,BL,19460.162789,167.69,80.15,15.161868,434.258065,362.000000
559,pieve del grappa,26096,TV,18093.788296,37.34,175.74,15.161868,434.258065,300.000000
560,valbrenta,24125,VI,17956.865122,93.37,53.28,15.161868,434.258065,160.000000
561,colceresa,24126,VI,19783.550410,19.40,307.63,15.161868,434.258065,150.000000


## River density

In [453]:
df_river_density = pd.read_excel('./data/codiceISTAT_schedaLR14_2017.ods', 'ISTAT')
#df_density = df_density[['ProCom', 'Densità']]

In [454]:
df_river_density.head()

,cod. comune,comune,Provincia,Superficie Territoriale,Alta Tens. Abitativa,Cl. Sismica,Zona Altimetrica,Com.Litoraneo,m2 classe 1.1,m2 classe 1.2,m2 classe 1.3,m2 classe 1.4,classe_ 2,classe_3,classe_4,classe_5
0,23001,Affi,Verona,9.799115e+06,NO,3,C,NO,9.004371e+05,1.184847e+06,111488.771911,94334.549293,4.988987e+06,2.519021e+06,0,0.000000
1,23002,Albaredo d'Adige,Verona,2.806956e+07,NO,3,P,NO,2.435867e+06,1.573058e+06,77486.309329,83747.221385,2.248276e+07,5.606045e+05,10028.032285,846009.657850
2,23003,Angiari,Verona,1.342630e+07,NO,4,P,NO,1.060541e+06,8.997925e+05,404694.873533,32775.778874,1.024552e+07,2.044952e+05,7131.331046,571346.642105
3,23004,Arcole,Verona,1.893525e+07,NO,3,P,NO,2.142434e+06,1.410993e+06,96287.506907,74127.854180,1.520780e+07,3.744673e+04,0,69497.743789
4,23005,Badia Calavena,Verona,2.674027e+07,NO,2,M,NO,1.087524e+06,5.244485e+05,54086.253086,14338.349984,9.144607e+06,1.575538e+07,0,159884.900683


In [455]:
df_river_density[RIVER_DENSITY] = df_river_density['classe_5'] / df_river_density['Superficie Territoriale']
df_river_density = df_river_density[['cod. comune', RIVER_DENSITY]]

In [456]:
df_river_density.head()

,cod. comune,River density
0,23001,0.000000
1,23002,0.030140
2,23003,0.042554
3,23004,0.003670
4,23005,0.005979


In [457]:
municipality_info_df = pd.merge(
    municipality_info_df, 
    df_river_density, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='cod. comune', 
    how = 'left'
)

municipality_info_df.drop('cod. comune', axis=1, inplace=True)

In [458]:
municipality_info_df.head()

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density
0,abano terme,28001,PD,22672.425104,21.57,946.5,31.767918,70.0,13.535090,0.003281
1,adria,29001,RO,19309.673843,113.52,170.6,15.708153,11.0,1.057863,0.043392
2,affi,23001,VR,20106.002743,9.85,239.1,17.881218,248.0,231.337300,0.000000
3,agna,28002,PD,17973.677643,18.83,171.8,10.466909,3.0,2.337880,0.012827
4,agordo,25001,BL,23715.945362,23.67,173.4,6.942357,2329.0,1267.740000,0.004915


In [459]:
municipality_info_df[municipality_info_df[RIVER_DENSITY].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density
556,barbarano mossano,24124,VI,18563.440208,33.486,191.80,15.161868,430.000000,72.0,NaN
557,borgo veneto,28107,PD,17286.636487,39.171,178.70,15.161868,15.000000,12.0,NaN
558,borgo valbelluna,25074,BL,19460.162789,167.690,80.15,15.161868,434.258065,362.0,NaN
559,pieve del grappa,26096,TV,18093.788296,37.340,175.74,15.161868,434.258065,300.0,NaN
560,valbrenta,24125,VI,17956.865122,93.370,53.28,15.161868,434.258065,160.0,NaN
561,colceresa,24126,VI,19783.550410,19.400,307.63,15.161868,434.258065,150.0,NaN
562,lusiana conco,24127,VI,17003.585115,61.190,76.32,15.161868,434.258065,830.0,NaN


In [460]:
municipality_info_df[RIVER_DENSITY].fillna(municipality_info_df[RIVER_DENSITY].mean(), inplace=True)

In [461]:
municipality_info_df[municipality_info_df[RIVER_DENSITY].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density


In [462]:
municipality_info_df[RIVER_DENSITY].mean()

0.02204408185997145

## Rainfall Frequency

In [463]:
df_rainfall = pd.read_excel('./data/Tavole_dati_meteo_2019_capoluoghi-provincia.xlsx', 
                            'Tavola_4',  skiprows = 2, header=[0,1], na_values='....')
#df_density = df_density[['ProCom', 'Densità']]

In [464]:
df_rainfall = df_rainfall[df_rainfall['COMUNI']['Unnamed: 0_level_1'].isin(provinces)]

In [465]:
df_rainfall.head()

COMUNI                    Giorni senza pioggia\nR0  \
   Unnamed: 0_level_1 differenza 2019\ndal valore medio 2007-2016   
23             Verona                                         7.4   
24            Vicenza                                         NaN   
25            Belluno                                         2.9   
26            Treviso                                         0.7   
27            Venezia                                           1   

                                Giorni con \nprecipitazione >1 mm \nR1  \
   valore medio\n2007-2016 differenza 2019\ndal valore medio 2007-2016   
23                   274.6                                        -3.1   
24                     NaN                                         NaN   
25                   248.1                                        -3.2   
26                   270.3                                        -0.4   
27                   290.0                                       -0.75   

                               Giorni con \nprecipitazione >10 mm\nR10  \
   valore medio\n2007-2016 differenza 2019\ndal valore medio 2007-2016   
23                   86.10                                         5.7   
24                     NaN                                         NaN   
25                  117.20                                         5.3   
26                   94.40                                         8.8   
27                   74.75                                         4.1   

                              Giorni con \nprecipitazione >20 mm \nR20  \
   valore medio\n2007-2016 differenza 2019\ndal valore medio 2007-2016   
23                    28.3                                        -0.2   
24                     NaN                                         NaN   
25                    51.7                                        10.8   
26                    38.2                                         7.6   
27                    23.9                                        1.65   

                               Giorni con \nprecipitazione >50 mm\nR50  \
   valore medio\n2007-2016 differenza 2019\ndal valore medio 2007-2016   
23                   11.20                                         0.9   
24                     NaN                                         NaN   
25                   26.20                                         4.1   
26                   19.40                                        -0.4   
27                    8.35                                           0   

                                 Giorni consecutivi \ncon pioggia\nCWD  \
   valore medio\n2007-2016 differenza 2019\ndal valore medio 2007-2016   
23                     1.1                                         3.4   
24                     NaN                                         NaN   
25                     4.9                                          -3   
26                     2.4                                        -0.7   
27                     1.0                                         3.8   

                               Giorni consecutivi\n senza pioggia\nCDD  \
   valore medio\n2007-2016 differenza 2019\ndal valore medio 2007-2016   
23                     5.6                                        -6.5   
24                     NaN                                         NaN   
25                     8.0                                          -6   
26                     6.7                                        -7.2   
27                     5.2                                           3   

                           Intensità di pioggia giornaliera \nSDII (mm)  \
   valore medio\n2007-2016  differenza 2019\ndal valore medio 2007-2016   
23                    24.5                                         1.16   
24                     NaN                                          NaN   
25                    24.0                                         4.18   
26                    24.2                

In [466]:
df_rainfall = pd.concat([
    df_rainfall.xs('COMUNI', axis=1, level=0, drop_level=True),
    df_rainfall.xs('valore medio\n2007-2016', axis=1, level=1, drop_level=True)
], axis=1)

In [467]:
#df_rainfall.set_index('Unnamed: 0_level_1', inplace=True)

In [468]:
df_rainfall

,Unnamed: 0_level_1,Giorni senza pioggia\nR0,Giorni con \nprecipitazione >1 mm \nR1,Giorni con \nprecipitazione >10 mm\nR10,Giorni con \nprecipitazione >20 mm \nR20,Giorni con \nprecipitazione >50 mm\nR50,Giorni consecutivi \ncon pioggia\nCWD,Giorni consecutivi\n senza pioggia\nCDD,Intensità di pioggia giornaliera \nSDII (mm)
23,Verona,274.6,86.10,28.3,11.20,1.1,5.6,24.5,9.490
24,Vicenza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Belluno,248.1,117.20,51.7,26.20,4.9,8.0,24.0,13.700
26,Treviso,270.3,94.40,38.2,19.40,2.4,6.7,24.2,12.360
27,Venezia,290.0,74.75,23.9,8.35,1.0,5.2,27.0,9.155
28,Padova,276.1,87.40,34.7,15.30,1.4,5.3,24.2,11.230
29,Rovigo,285.3,77.90,24.8,8.80,0.5,4.9,27.1,9.270


In [469]:
#df_rainfall[df_rainfall['Unnamed: 0_level_1'].isin(['Verona', 'Padova'])]


In [470]:
# Fill Vicenza with mean values among Padova and Verona
df_rainfall.fillna(
    df_rainfall[df_rainfall['Unnamed: 0_level_1'].isin(['Verona', 'Padova'])].drop('Unnamed: 0_level_1', axis=1).mean(), 
    inplace=True
)

In [471]:
def get_rainfall_frequency(province):
    return 365 - df_rainfall[df_rainfall['Unnamed: 0_level_1'] == provinces_map[province]]['Giorni senza pioggia\nR0'].sum()
    
def get_rainfall_amount(province):
    return df_rainfall[
        df_rainfall['Unnamed: 0_level_1'] == provinces_map[province]
    ]['Intensità di pioggia giornaliera \nSDII (mm)'].sum()

In [472]:
#municipality_info_df[RAINFALL_FREQUENCY] = 0
municipality_info_df[RAINFALL_FREQUENCY] = municipality_info_df.apply(
    lambda row: get_rainfall_frequency(row[PROVINCE_COLUMN]), axis=1
)

municipality_info_df[RAINFALL_AMOUNT] = municipality_info_df.apply(
    lambda row: get_rainfall_amount(row[PROVINCE_COLUMN]) + row[RIVER_DENSITY]**2, axis=1
)

In [473]:
municipality_info_df.head()

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density,Rainfall frequency,Rainfall amount
0,abano terme,28001,PD,22672.425104,21.57,946.5,31.767918,70.0,13.535090,0.003281,88.9,11.230011
1,adria,29001,RO,19309.673843,113.52,170.6,15.708153,11.0,1.057863,0.043392,79.7,9.271883
2,affi,23001,VR,20106.002743,9.85,239.1,17.881218,248.0,231.337300,0.000000,90.4,9.490000
3,agna,28002,PD,17973.677643,18.83,171.8,10.466909,3.0,2.337880,0.012827,88.9,11.230165
4,agordo,25001,BL,23715.945362,23.67,173.4,6.942357,2329.0,1267.740000,0.004915,116.9,13.700024


## Flood

In [474]:
flood_df = pd.read_excel('./data/Indicatori_Regione_Veneto.xlsx', decimal=',')
flood_df.head()

,DATA_RIF,ID_TERRITORIO_IND,ID_REGIONE,DZREG,CODPRO,DZPRO,PROCOM,DZCOM,LIT,LIT_TXT,...,TOT_ARCHEO_R,TOT_PARC_GIAR,TOT_PARC_GIAR_P,TOT_PARC_GIAR_R,PRO_COM_110,ERECQ,ERECQ_TXT,ERECEM,ERESPP,EREMP
0,2018-01-01,11235,5,Veneto,23,Verona,23001,Affi,0,non litoraneo,...,577,0,11,39,23001,6,entro il 1990,6,3,2
1,2018-01-01,11236,5,Veneto,23,Verona,23002,Albaredo d'Adige,0,non litoraneo,...,577,0,11,39,23002,5,entro il 1980,5,3,2
2,2018-01-01,11237,5,Veneto,23,Verona,23003,Angiari,0,non litoraneo,...,577,0,11,39,23003,7,entro il 2000,7,3,2
3,2018-01-01,11238,5,Veneto,23,Verona,23004,Arcole,0,non litoraneo,...,577,0,11,39,23004,5,entro il 1980,5,2,2
4,2018-01-01,11239,5,Veneto,23,Verona,23005,Badia Calavena,0,non litoraneo,...,577,0,11,39,23005,6,entro il 1990,6,3,2


In [475]:
flood_df = flood_df[['PROCOM', 'IDR_AREAP1', 'IDR_AREAP2', 'IDR_AREAP3']]

In [476]:
flood_df.head()

,PROCOM,IDR_AREAP1,IDR_AREAP2,IDR_AREAP3
0,23001,0.277409,0.219825,0.137521
1,23002,0.000041,0.000041,0.000000
2,23003,0.000000,0.000000,0.000000
3,23004,3.464366,3.277428,2.931939
4,23005,0.000000,0.000000,0.000000


In [477]:
#.replace(0, flood_df[].min(axis=0), inplace=True)

In [478]:
flood_df[FLOOD] = flood_df['IDR_AREAP1'] +  flood_df['IDR_AREAP2'] + flood_df['IDR_AREAP3']
flood_df.drop(['IDR_AREAP1', 'IDR_AREAP2', 'IDR_AREAP3'], axis=1, inplace=True)

In [479]:
flood_df

,PROCOM,Flood
0,23001,0.634755
1,23002,0.000082
2,23003,0.000000
3,23004,9.673733
4,23005,0.000000
...,...,...
566,29048,32.074036
567,29049,14.151275
568,29050,11.725761
569,29051,21.067430


In [480]:
municipality_info_df = pd.merge(
    municipality_info_df, 
    flood_df, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='PROCOM', 
    how = 'left'
)

municipality_info_df.drop('PROCOM', axis=1, inplace=True)

In [481]:
municipality_info_df[FLOOD] = municipality_info_df[FLOOD] / municipality_info_df[SURFACE_COLUMN]

In [482]:
municipality_info_df.head()

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density,Rainfall frequency,Rainfall amount,Flood
0,abano terme,28001,PD,22672.425104,21.57,946.5,31.767918,70.0,13.535090,0.003281,88.9,11.230011,2.302738
1,adria,29001,RO,19309.673843,113.52,170.6,15.708153,11.0,1.057863,0.043392,79.7,9.271883,1.036059
2,affi,23001,VR,20106.002743,9.85,239.1,17.881218,248.0,231.337300,0.000000,90.4,9.490000,0.064442
3,agna,28002,PD,17973.677643,18.83,171.8,10.466909,3.0,2.337880,0.012827,88.9,11.230165,0.998627
4,agordo,25001,BL,23715.945362,23.67,173.4,6.942357,2329.0,1267.740000,0.004915,116.9,13.700024,0.000000


In [483]:
#unicipality_info_df[FLOOD] = municipality_info_df[FLOOD] / municipality_info_df[SURFACE_COLUMN]

In [484]:
municipality_info_df[municipality_info_df[FLOOD].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density,Rainfall frequency,Rainfall amount,Flood
558,borgo valbelluna,25074,BL,19460.162789,167.69,80.15,15.161868,434.258065,362.0,0.022044,116.90,13.700486,NaN
559,pieve del grappa,26096,TV,18093.788296,37.34,175.74,15.161868,434.258065,300.0,0.022044,94.70,12.360486,NaN
560,valbrenta,24125,VI,17956.865122,93.37,53.28,15.161868,434.258065,160.0,0.022044,89.65,10.360486,NaN
561,colceresa,24126,VI,19783.550410,19.40,307.63,15.161868,434.258065,150.0,0.022044,89.65,10.360486,NaN
562,lusiana conco,24127,VI,17003.585115,61.19,76.32,15.161868,434.258065,830.0,0.022044,89.65,10.360486,NaN


In [485]:
municipality_info_df[FLOOD].fillna(municipality_info_df[FLOOD].mean(), inplace=True)

In [486]:
municipality_info_df[municipality_info_df[FLOOD].isna()]

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density,Rainfall frequency,Rainfall amount,Flood


# Probabilities

In [488]:
from sklearn.preprocessing import MinMaxScaler

columns_to_scale = [PER_UNIT_GDP, POPULATION_DENSITY, ROAD_DENSITY, SLOPE, ELEVATION, RIVER_DENSITY, 
                    RAINFALL_FREQUENCY, RAINFALL_AMOUNT, FLOOD]

scaler = MinMaxScaler()

df_scaled = municipality_info_df.copy()

df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])


In [489]:
df_scaled

,Municipality Name,Istat code,Province,Per unit GDP,Surface (km2),Population density,Road density,Slope,Elevation,River density,Rainfall frequency,Rainfall amount,Flood
0,abano terme,28001,PD,0.741966,21.57,0.415974,0.471708,0.026004,0.006801,0.004605,0.331742,0.455822,0.885722
1,adria,29001,RO,0.526982,113.52,0.072837,0.223444,0.003442,0.000358,0.060901,0.112172,0.025676,0.398508
2,affi,23001,VR,0.577892,9.85,0.103131,0.257037,0.094073,0.119265,0.000000,0.367542,0.073590,0.024787
3,agna,28002,PD,0.441570,18.83,0.073368,0.142422,0.000382,0.001019,0.018003,0.331742,0.455856,0.384110
4,agordo,25001,BL,0.808679,23.67,0.074076,0.087937,0.889866,0.654421,0.006898,1.000000,0.998415,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,borgo valbelluna,25074,BL,0.536603,167.69,0.032837,0.215000,0.165299,0.186734,0.030939,1.000000,0.998516,0.142315
559,pieve del grappa,26096,TV,0.449249,37.34,0.075111,0.215000,0.165299,0.154720,0.030939,0.470167,0.704156,0.142315
560,valbrenta,24125,VI,0.440495,93.37,0.020953,0.215000,0.165299,0.082429,0.030939,0.349642,0.264812,0.142315
561,colceresa,24126,VI,0.557277,19.40,0.133438,0.215000,0.165299,0.077266,0.030939,0.349642,0.264812,0.142315


In [549]:
per_unit_gdp_conditions = [
    (df_scaled[PER_UNIT_GDP] >= 0.6),
    (df_scaled[PER_UNIT_GDP] > 0.4) & (df_scaled[PER_UNIT_GDP] < 0.6),
    (df_scaled[PER_UNIT_GDP] <= 0.4)
]

per_unit_gdp_df = pd.DataFrame(
    {'': [len(df_scaled[c]) / df_scaled.shape[0] for c in per_unit_gdp_conditions]},
    index=['{} ({})'.format(PER_UNIT_GDP, v) for v in ['High', 'Medium', 'Low']]
)

In [550]:
per_unit_gdp_df

,
Per unit GDP (High),0.325044
Per unit GDP (Medium),0.531083
Per unit GDP (Low),0.143872


In [541]:
df_scaled[POPULATION_DENSITY].mean()

0.12980078796244254

In [575]:
from itertools import product

In [576]:
population_density_conditions = [
    (df_scaled[POPULATION_DENSITY] >= 0.2),
    (df_scaled[POPULATION_DENSITY] > 0.05) & (df_scaled[POPULATION_DENSITY] < 0.2),
    (df_scaled[POPULATION_DENSITY] <= 0.05)
]

cols = pd.MultiIndex.from_product([['{} ({})'.format(PER_UNIT_GDP, v) for v in ['High', 'Medium', 'Low']]])

population_density_df = pd.DataFrame(
    [ [len(df_scaled[c & c_p]) / df_scaled[c].shape[0] for c in per_unit_gdp_conditions] for c_p in population_density_conditions
        #[len(df_scaled[c[0] & c[1]]) / df_scaled[c[0]].shape[0] for c in combinations([p_c] + per_unit_gdp_conditions, 2)] for p_c in population_density_conditions
    ],
    index=['{} ({})'.format(POPULATION_DENSITY, v) for v in ['Dense', 'Medium', 'Sparse']],
    columns=cols
)

In [577]:
population_density_df

,Per unit GDP (High),Per unit GDP (Medium),Per unit GDP (Low)
Population density (Dense),0.491803,0.090301,0.012346
Population density (Medium),0.409836,0.628763,0.283951
Population density (Sparse),0.098361,0.280936,0.703704


In [578]:
population_density_df

,Per unit GDP (High),Per unit GDP (Medium),Per unit GDP (Low)
Population density (Dense),0.491803,0.090301,0.012346
Population density (Medium),0.409836,0.628763,0.283951
Population density (Sparse),0.098361,0.280936,0.703704


In [579]:
df_scaled[ROAD_DENSITY].mean()

0.21499958853018786

In [584]:
road_density_conditions = [
    (df_scaled[ROAD_DENSITY] >= 0.3),
    (df_scaled[ROAD_DENSITY] > 0.15) & (df_scaled[ROAD_DENSITY] < 0.3),
    (df_scaled[ROAD_DENSITY] <= 0.15)
]

cols = pd.MultiIndex.from_product([
    ['{} ({})'.format(PER_UNIT_GDP, v) for v in ['High', 'Medium', 'Low']],
    ['{} ({})'.format(POPULATION_DENSITY, v) for v in ['Dense', 'Medium', 'Sparse']],
])

#print(cols)

evidence_values = []
for v in road_density_conditions:
    evidence_for_v = []
    for c in product(per_unit_gdp_conditions, population_density_conditions):
        evidence_for_v.append(len(df_scaled[c[0] & c[1] & v]) / max(df_scaled[c[0] & c[1]].shape[0], 1))
    evidence_values.append(evidence_for_v)

# print(evidence_values)    

road_density_df = pd.DataFrame(
    evidence_values,
    index=['{} ({})'.format(POPULATION_DENSITY, v) for v in ['Dense', 'Medium', 'Sparse']],
    columns=cols
)

[[0.37777777777777777, 0.08, 0.0, 0.25925925925925924, 0.07446808510638298, 0.03571428571428571, 1.0, 0.043478260869565216, 0.017543859649122806], [0.5888888888888889, 0.8266666666666667, 0.0, 0.7407407407407407, 0.8617021276595744, 0.47619047619047616, 0.0, 0.8695652173913043, 0.543859649122807], [0.03333333333333333, 0.09333333333333334, 1.0, 0.0, 0.06382978723404255, 0.4880952380952381, 0.0, 0.08695652173913043, 0.43859649122807015]]


In [585]:
road_density_df

Per unit GDP (High)  \
                            Population density (Dense)   
Population density (Dense)                    0.377778   
Population density (Medium)                   0.588889   
Population density (Sparse)                   0.033333   

                                                         \
                            Population density (Medium)   
Population density (Dense)                     0.080000   
Population density (Medium)                    0.826667   
Population density (Sparse)                    0.093333   

                                                         \
                            Population density (Sparse)   
Population density (Dense)                          0.0   
Population density (Medium)                         0.0   
Population density (Sparse)                         1.0   

                                 Per unit GDP (Medium)  \
                            Population density (Dense)   
Population density (Dense)                    0.259259   
Population density (Medium)                   0.740741   
Population density (Sparse)                   0.000000   

                                                         \
                            Population density (Medium)   
Population density (Dense)                     0.074468   
Population density (Medium)                    0.861702   
Population density (Sparse)                    0.063830   

                                                         \
                            Population density (Sparse)   
Population density (Dense)                     0.035714   
Population density (Medium)                    0.476190   
Population density (Sparse)                    0.488095   

                                    Per unit GDP (Low)  \
                            Population density (Dense)   
Population density (Dense)                         1.0   
Population density (Medium)                        0.0   
Population density (Sparse)                        0.0   

                                                         \
                            Population density (Medium)   
Population density (Dense)                     0.043478   
Population density (Medium)                    0.869565   
Population density (Sparse)                    0.086957   

                                                         
                            Population density (Sparse)  
Population density (Dense)                     0.017544  
Population density (Medium)                    0.543860  
Population density (Sparse)                    0.438596

In [ ]:
"""
PER_UNIT_GDP: [
        [0.3], 
        [0.63], 
        [0.07]
    ],
    POPULATION_DENSITY: [
        [0.7, 0.3, 0.05],
        [0.2, 0.55, 0.25],
        [0.1, 0.15, 0.7],
    ],
    ROAD_DENSITY: [
        [0.8, 0.7, 0.1, 0.6, 0.4, 0.1, 0.1, 0.05, 0.01],
        [0.19, 0.25, 0.35, 0.3, 0.5, 0.25, 0.25, 0.2, 0.15],
        [0.01, 0.05, 0.55, 0.1, 0.1, 0.65, 0.65, 0.75, 0.84]
    ],
    ELEVATION: [
        [0.15],
        [0.1], 
        [0.75]
    ],
    SLOPE: [
        [0.75, 0.6, 0.05], 
        [0.25, 0.4, 0.95]
    ],
    RAINFALL_FREQUENCY: [
        [0.3], 
        [0.6], 
        [0.1]
    ],
    RIVER_DENSITY: [
        [0.4], 
        [0.6]
    ],
    RAINFALL_AMOUNT: [
        [0.7, 0.5, 0.55, 0.3, 0.1, 0.01],
        [0.2, 0.25, 0.3, 0.4, 0.3, 0.04],
        [0.1, 0.25, 0.15, 0.3, 0.6, 0.95]
    ],
    FLOOD: [
        [0.07, 0.03, 0.005, 0.2, 0.1, 0.008, 0.05, 0.009, 0.0005, 0.13, 0.08, 0.006, 0.008, 0.002, 0.0001, 0.1, 0.04, 0.0002],
        [0.93, 0.97, 0.995, 0.8, 0.9, 0.992, 0.95, 0.991, 0.9995, 0.87, 0.92, 0.994, 0.992, 0.998, 0.9999, 0.9, 0.96, 0.9998]
    ],
""";